# Getting Started with SingleStoreDB and Python

This notebook is a simple demonstration of the SingleStore Python package capabilities. It is by no means exhaustive of all features, but is 
meant as an introduction to the types of things you can do. There are three Python packages utilized which represent layers of capabilities
that build on each other. The three packages are:

* SingleStoreDB Python - the base layer which implements the [Python DB-API](https://peps.python.org/pep-0249/) as well as the 
  [SingleStoreDB Cloud Management API](https://docs.singlestore.com/managed-service/en/user-and-workspace-administration/management-api.html).
* SingleStoreDB SQLAlchemy Dialect - this is a [SQLAlchemy](https://www.sqlalchemy.org) plugin which allows you to interact with SingleStoreDB using SQLAlchemy.
* SingleStoreDB Ibis Backend - this package implements a backend for the [Ibis](https://ibis-project.org/) framework which contains a generalized DataFrame
  API similar to the [pandas](https://pandas.pydata.org) DataFrame API that works with databases, data warehouses, and other backends (such as pandas itself).
  
Since the SingleStoreDB Ibis backend is built on the SingleStore SQLAlchemy dialect which is in turn built on the SingleStoreDB Python package,
we'll start right out using Ibis to exercise all of them at once.

In [1]:
import sys
sys.path.insert(0, '/home/ksmith/src/singlestoredb-python')

In [2]:
import ibis
import pandas as pd

We'll set up a few options to make the notebook nicer to use, including the display width of DataFrames and enabling the interactive features
of the Ibis package. The primary purpose of the `interactive` option is so that you don't have to manually run the `execute` method on expressions
to see the results. In our notebook, the `execute` method will be called implicitly each time an Ibis expression is rendered by the notebook.

In [3]:
pd.options.display.max_colwidth = 120
ibis.options.verbose = False
ibis.options.interactive = True

## Make a connection using Ibis

Connections to Ibis are made through the `ibis.singlestoredb` method. The connection parameters can be given using either the suggested Python DB-API
parameters (i.e., host=, user=, port=, etc.) or as a SQLAlchemy URL (e.g., user:password@host:port/database). In this case, we are getting the username and password from the environment variable `SINGLESTOREDB_USER` and `SINGLESTOREDB_PASSWORD`.

In [5]:
conn = ibis.singlestoredb.connect('root:@127.0.0.1/x_db')

SingleStoreDB also has an HTTP API which is wrapped in a Python DB-API compliant form, so it can be used as well by specifying an HTTP(s) URL and
the corresponding port number.

In [6]:
# conn = ibis.singlestoredb.connect('http://127.0.0.1:8000/x_db')

## Phone review data example

Our first example will use customer review data for phones. It includes information such as a reviewer name, the text of the review, a short summary
of the review, a numeric rating from 1 to 5, and the date of the review. We'll first read the data into a local pandas DataFrame to do some
cleanup, then upload it to the database to demonstrate the SingleStoreDB package capabilities

### Read sample data into pandas DataFrame

Here we're simply displaying the top of the CSV file containing the data to see the column names and the first line of data.

In [7]:
%%sh 

head -2 phones.csv

reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
"A30TL5EWN6DFXT","120401325X","christina","[0, 0]","They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again","4.0","Looks Good","1400630400","05 21, 2014"


The data is read using the `read_csv` function.

In [8]:
phones_df = pd.read_csv('phones.csv')
phones_df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept pop...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they do. They stick on great and they stay on the phone. They are super sty...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a yea...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"


We can see the column types using the `info` method. As you can see, there are two date columns, neither of which is in a convenient
format for use as a date. We'll first create an actual datetime column from those and drop the original columns.

In [9]:
phones_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      3921 non-null   object 
 1   asin            3921 non-null   object 
 2   reviewerName    3921 non-null   object 
 3   helpful         3921 non-null   object 
 4   reviewText      3921 non-null   object 
 5   overall         3921 non-null   float64
 6   summary         3920 non-null   object 
 7   unixReviewTime  3921 non-null   int64  
 8   reviewTime      3921 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 275.8+ KB


### Upload the data to the server

Now that we have some data to work with, we can upload it to the server using the `create_table` method. This method allows you to upload
data from a local DataFrame or materialize a table from an Ibis table expression.

In [10]:
conn.create_table?

Signature:
conn.create_table(
    name: 'str',
    expr: 'pd.DataFrame | ir.TableExpr | None' = None,
    schema: 'sch.Schema | None' = None,
    database: 'str | None' = None,
    force: 'bool' = False,
    storage_type: 'Optional[str]' = None,
) -> 'None'
Docstring:
Create a new table.

Parameters
----------
name
    Name of the new table.
expr
    An Ibis table expression or pandas DataFrame that will be used to
    extract the schema and the data of the new table. If not provided,
    `schema` must be given.
schema
    The schema for the new table. Only one of `schema` or `expr` can be
    provided.
database
    Name of the database where the table will be created, if not the
    default.
force
    Check whether a table exists before creating it
storage_type
    The storage type of table to create: COLUMNSTORE or ROWSTORE
File:      ~/.pyenv/versions/3.9.4/lib/python3.9/site-packages/ibis_singlestoredb/__init__.py
Type:      method


The code below will create the table in the server from the given DataFrame. The data types for the database table will be inferred from
the DataFrame data types. The `force=True` is merely used here in case you run the notebook code multiple times against the same
database. It will cause an existing `phones` table to be overwritten if it already exists.

In [11]:
phones_tbl = conn.create_table('phones', phones_df, force=True)
phones_tbl.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept pop...,4.0,Looks Good,1400630400,"05 21, 2014"
1,APX47D16JOP7H,120401325X,RLH,"[1, 2]",These make using the home button easy. My daughter and I both like them. I would purchase them again. Well worth th...,3.0,Cute,1381536000,"10 12, 2013"
2,A6FGO4TBZ3QFZ,3998899561,Abdullah Albyati,"[1, 2]",it worked for the first week then it only charge my phone to 20%. it is a waste of money.,1.0,not a good Idea,1384992000,"11 21, 2013"


We can get information about the table in the server by using the `info` method, just as you would with a local pandas DataFrame.

In [12]:
phones_tbl.info()

                       Summary of phones                       
                           3921 rows                           
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Name           ┃ Type                   ┃ # Nulls ┃ % Nulls ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ reviewerID     │ String(nullable=True)  │       0 │    0.00 │
│ asin           │ String(nullable=True)  │       0 │    0.00 │
│ reviewerName   │ String(nullable=True)  │       0 │    0.00 │
│ helpful        │ String(nullable=True)  │       0 │    0.00 │
│ reviewText     │ String(nullable=True)  │       0 │    0.00 │
│ overall        │ Float64(nullable=True) │       0 │    0.00 │
│ summary        │ String(nullable=True)  │       1 │    0.03 │
│ unixReviewTime │ Int64(nullable=True)   │       0 │    0.00 │
│ reviewTime     │ String(nullable=True)  │       0 │    0.00 │
└────────────────┴────────────────────────┴─────────┴─────────┘


Here is the `CREATE TABLE` statement resulting from the data upload step. The `show` attribute of the connection gives access to all of
the `SHOW` commands in the database such as `functions`, `create_function`, `status`, `databases`, and so on.

In [13]:
conn.show.create_table('phones')

Name,CreateTable
phones,"CREATE TABLE `phones` ( `reviewerID` text CHARACTER SET utf8 COLLATE utf8_general_ci, `asin` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewerName` text CHARACTER SET utf8 COLLATE utf8_general_ci, `helpful` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewText` text CHARACTER SET utf8 COLLATE utf8_general_ci, `overall` float DEFAULT NULL, `summary` text CHARACTER SET utf8 COLLATE utf8_general_ci, `unixReviewTime` bigint(20) DEFAULT NULL, `reviewTime` text CHARACTER SET utf8 COLLATE utf8_general_ci, KEY `__UNORDERED` () USING CLUSTERED COLUMNSTORE , SHARD KEY () ) AUTOSTATS_CARDINALITY_MODE=OFF AUTOSTATS_HISTOGRAM_MODE=OFF AUTOSTATS_SAMPLING=OFF SQL_MODE='STRICT_ALL_TABLES'"


### Convert review date / time columns

This data set has a less than optimal way of storing the dates of the reviews. It has two columns that contain the same information, and neither
of them is convenient to work with. Let's create a new column that contains the date in a timestamp column. To do that, we simply cast the
existing `unixReviewTime` column to a timestamp type. We'll also use the `name` method to assign a name to the resulting column expression.

In [14]:
date = phones_tbl.unixReviewTime.cast('timestamp').name('date')
date

,date
0,2014-05-21
1,2013-10-12
2,2013-11-21
3,2014-04-03
4,2014-04-13
...,...
3916,2013-08-15
3917,2012-12-29
3918,2010-08-10
3919,2013-02-17


The `overall` rating column was uploaded as floating point data, but it only contains integers. Let's cast that to an integer.

In [15]:
overall = phones_tbl.overall.cast('int').name('overall')
overall

,overall
0,5
1,5
2,5
3,5
4,5
...,...
3916,5
3917,5
3918,3
3919,5


Now that we have our two processed columns, we can remove the old columns we don't need anymore and insert our
new columns expressions with the cleaned data.

In [16]:
columns = [x for x in phones_tbl.columns if 'Time' not in x and x != 'overall'] + [overall, date]

In [17]:
phones_tbl = phones_tbl[columns] 
phones_tbl

,reviewerID,asin,reviewerName,helpful,reviewText,summary,overall,date
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept pop...,Looks Good,4,2014-05-21
1,APX47D16JOP7H,120401325X,RLH,"[1, 2]",These make using the home button easy. My daughter and I both like them. I would purchase them again. Well worth th...,Cute,3,2013-10-12
2,A6FGO4TBZ3QFZ,3998899561,Abdullah Albyati,"[1, 2]",it worked for the first week then it only charge my phone to 20%. it is a waste of money.,not a good Idea,1,2013-11-21
3,A8AJS1DW7L3JJ,3998899561,Agata Majchrzak,"[1, 1]","This is a fantastic case. Very stylish and protects my phone. Easy access to all buttons and features, without any l...",Perfect Case,5,2014-04-03
4,A2YO4SCWAWNYBI,3998899561,Alex Maslakov,"[0, 0]",this case fits perfectly on the s4 and keeps me powerd all day I can't complain! a+ recommend it to all,Just what I needed,5,2014-04-13
...,...,...,...,...,...,...,...,...
3916,A3FCCLICKZSAJP,B000TUP97A,RGW,"[0, 0]","I had an old Motorola Razor cell phone, but it would not charge. I tried this battery because the price was right a...",Did the trick,5,2013-08-15
3917,A3D035KQL9KQFK,B000TUP97A,Shawn Stiner,"[0, 0]",Looking for an OEM replacement...this is it. Works just as good as the orginal battery maybe even better because it ...,OEM,5,2012-12-29
3918,ASF3J4UYV34KA,B000TUWHFM,Average Customer,"[0, 0]",The power retention of this battery is half as much as the original battery. Many of the replacement batteries do n...,Good but not Great,3,2010-08-10
3919,A18TW6ZA16VUM2,B000TUWHFM,"David Taber, SalesLogistix CEO www.SFDC-secr...","[0, 0]","This is the original equipment OEM battery for Razors and other vintage motorola phones. works great, charges quick...","Great battery, great price",5,2013-02-17


The `phones_tbl` object now contains both references to actual columns in the `phones` table as well as computed expressions.
It can be thought of as a client-side view object, but still works the same way as our original table. We can use the `info`
method to show the schema information.

In [18]:
phones_tbl.info()

                             Summary of Selection                             
                                  3921 rows                                   
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Name         ┃ Type                                    ┃ # Nulls ┃ % Nulls ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ reviewerID   │ String(nullable=True)                   │       0 │    0.00 │
│ asin         │ String(nullable=True)                   │       0 │    0.00 │
│ reviewerName │ String(nullable=True)                   │       0 │    0.00 │
│ helpful      │ String(nullable=True)                   │       0 │    0.00 │
│ reviewText   │ String(nullable=True)                   │       0 │    0.00 │
│ summary      │ String(nullable=True)                   │       1 │    0.03 │
│ overall      │ Int64(nullable=True)                    │       0 │    0.00 │
│ date         │ Timestamp(timezone=None, nullable=T

It's also possible to access the schema information for traversing progamatically using the `schema` method of the table.

In [19]:
phones_tbl.schema()

ibis.Schema {
  reviewerID    string
  asin          string
  reviewerName  string
  helpful       string
  reviewText    string
  summary       string
  overall       int64
  date          timestamp
}

### Create a table from a table expression

As mentioned earlier, the `create_table` method can be used to create tables in the database from Ibis table
expressions. That means that we can materialize our new table into the database if we so desire. This simply
selects data from an expression into a new table, no work is being done on the client side.

In [20]:
tbl_2 = conn.create_table('phones2', phones_tbl, force=True)
tbl_2

,reviewerID,asin,reviewerName,helpful,reviewText,summary,overall,date
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept pop...,Looks Good,4,2014-05-21
1,APX47D16JOP7H,120401325X,RLH,"[1, 2]",These make using the home button easy. My daughter and I both like them. I would purchase them again. Well worth th...,Cute,3,2013-10-12
2,A6FGO4TBZ3QFZ,3998899561,Abdullah Albyati,"[1, 2]",it worked for the first week then it only charge my phone to 20%. it is a waste of money.,not a good Idea,1,2013-11-21
3,A8AJS1DW7L3JJ,3998899561,Agata Majchrzak,"[1, 1]","This is a fantastic case. Very stylish and protects my phone. Easy access to all buttons and features, without any l...",Perfect Case,5,2014-04-03
4,A2YO4SCWAWNYBI,3998899561,Alex Maslakov,"[0, 0]",this case fits perfectly on the s4 and keeps me powerd all day I can't complain! a+ recommend it to all,Just what I needed,5,2014-04-13
...,...,...,...,...,...,...,...,...
3916,A3ON3XWGRCIS4I,B000TLVMMA,A. LANE,"[1, 2]","I got this sim card as a free promo item with the purchase of a cell phone, the rates for this service are really nu...",very overpriced,1,2009-10-22
3917,ABXANRX4GPYRZ,B000TLVMMA,hdtravel,"[0, 4]",This SIM is an excellent product to have with you and your unlocked Quad band GSM phone when you travel abroad.The c...,Good Deal for Emergency Use,4,2008-06-11
3918,A3UKWQS8SRW6IO,B000TLVMMA,OrchidSlayer,"[7, 9]",I have been using this card for several years. I frequently travel internationally and need short-term access to int...,Great for short term international use,5,2008-12-01
3919,ASM1YXKJZADC,B000TLVMMA,S. Alexander Suzanne Alexander,"[0, 0]",I bought the card and took it on an international trip. I put the card into my phoneand went to the site to activat...,"A great mystery, and not a good one.",1,2014-06-23


In [21]:
conn.show.tables()

Name
phones
phones2
titanic


In [22]:
tbl_2.info()

                              Summary of phones2                              
                                  3921 rows                                   
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Name         ┃ Type                                    ┃ # Nulls ┃ % Nulls ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ reviewerID   │ String(nullable=True)                   │       0 │    0.00 │
│ asin         │ String(nullable=True)                   │       0 │    0.00 │
│ reviewerName │ String(nullable=True)                   │       0 │    0.00 │
│ helpful      │ String(nullable=True)                   │       0 │    0.00 │
│ reviewText   │ String(nullable=True)                   │       0 │    0.00 │
│ summary      │ String(nullable=True)                   │       1 │    0.03 │
│ overall      │ Int64(nullable=True)                    │       0 │    0.00 │
│ date         │ Timestamp(timezone=None, nullable=T

In [23]:
conn.show.create_table('phones2')

Name,CreateTable
phones2,"CREATE TABLE `phones2` ( `reviewerID` text CHARACTER SET utf8 COLLATE utf8_general_ci, `asin` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewerName` text CHARACTER SET utf8 COLLATE utf8_general_ci, `helpful` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewText` text CHARACTER SET utf8 COLLATE utf8_general_ci, `summary` text CHARACTER SET utf8 COLLATE utf8_general_ci, `overall` bigint(20) DEFAULT NULL, `date` timestamp NULL DEFAULT NULL, KEY `__UNORDERED` () USING CLUSTERED COLUMNSTORE , SHARD KEY () ) AUTOSTATS_CARDINALITY_MODE=OFF AUTOSTATS_HISTOGRAM_MODE=OFF AUTOSTATS_SAMPLING=OFF SQL_MODE='STRICT_ALL_TABLES'"


For this example, we'll drop this table and use our original object.

In [24]:
conn.drop_table('phones2')

### DataFrame operations

Now that we have a table in the database and a DataFrame-like obect connected to it, let's do some operations with the data. For this example, 
we'll enable the `verbose` option in Ibis. This will display the SQL code that is sent to the server when evaluating the expression. In this 
example, we are building a sentence from various columns in the data table. We are using the reviwer name (which is capitalized), the summary,
and the overall rating (which is cast to a string). All of these are concatenated together using the `+` as you normally would when
concatenating strings in Python.

In [25]:
# Enable display of SQL that is executed in the background
ibis.options.verbose = True

texpr = phones_tbl.reviewerName.capitalize() + \
        ' says "' + \
        phones_tbl.summary + \
        '" and gives it ' + \
        phones_tbl.overall.cast('string') + \
        ' stars.'
texpr

SELECT concat(concat(concat(concat(concat(concat(ucase(left(t0.`reviewerName`, %(left_1)s)), substring(t0.`reviewerName`, %(substring_1)s)), %(param_1)s), t0.summary), %(param_2)s), CAST(t0.overall AS CHAR)), %(param_3)s) AS tmp 
FROM (SELECT t1.`reviewerID` AS `reviewerID`, t1.asin AS asin, t1.`reviewerName` AS `reviewerName`, t1.helpful AS helpful, t1.`reviewText` AS `reviewText`, t1.summary AS summary, CAST(t1.overall AS SIGNED INTEGER) AS overall, convert_tz(from_unixtime(t1.`unixReviewTime`), %(convert_tz_1)s, %(convert_tz_2)s) AS date 
FROM phones AS t1) AS t0 
 LIMIT %(param_4)s
SELECT concat(concat(concat(concat(concat(concat(ucase(left(t0.`reviewerName`, %(left_1)s)), substring(t0.`reviewerName`, %(substring_1)s)), %(param_1)s), t0.summary), %(param_2)s), CAST(t0.overall AS CHAR)), %(param_3)s) AS tmp 
FROM (SELECT t1.`reviewerID` AS `reviewerID`, t1.asin AS asin, t1.`reviewerName` AS `reviewerName`, t1.helpful AS helpful, t1.`reviewText` AS `reviewText`, t1.summary AS summary

,tmp
0,"Erica says ""LOVE LOVE LOVE"" and gives it 5 stars."
1,"Tyler Evans says ""best thing ever.."" and gives it 5 stars."
2,"Adam says ""Solid Case"" and gives it 5 stars."
3,"Tim g says ""Exactly what I needed"" and gives it 5 stars."
4,"Zonaldo Reefey Zonaldo Reefey says ""SUPER DUPER QUALITY!"" and gives it 5 stars."
...,...
3916,"A. LANE says ""very overpriced"" and gives it 1 stars."
3917,"Hdtravel says ""Good Deal for Emergency Use"" and gives it 4 stars."
3918,"OrchidSlayer says ""Great for short term international use"" and gives it 5 stars."
3919,"S. Alexander Suzanne Alexander says ""A great mystery, and not a good one."" and gives it 1 stars."


As you can see from the SQL output above, the sentence is constructed using `concat` in the query. Capitalization is done using `ucase` and `substring`. 
The `:param_#` portions are used by Ibis to inject literal Python values from the expression. And, you'll see that the result table contains a column
of string values that are the result of the expression.

#### String operations

There are many string operations available in Ibis. Many of them correspond to operations on pandas DataFrames and / or Python string methods.
The code below prints a list of all methods on string columns.

In [26]:
print(' '.join([x for x in dir(texpr) if not x.startswith('_')]))

approx_median approx_nunique arbitrary argmax argmin ascii_str between bottomk capitalize case cases cast coalesce collect compile concat contains convert_base count cume_dist cummax cummin dense_rank describe endswith equals execute fillna find find_in_set first get_name greatest group_concat has_name hash hashbytes identical_to ilike initcap isin isnull join lag last lead least left length like lower lpad lstrip max min name notin notnull nth ntile nullif nunique op over parent parse_url percent_rank pipe rank re_extract re_replace re_search repeat replace reverse right rlike rpad rstrip split startswith strip substitute substr summary to_projection to_timestamp topk translate type typeof upper value_counts verify visualize


#### Operations on other column types

Other data types have different methods that can be called. For example, this is the list of methods on date/time columns.

In [27]:
print(' '.join([x for x in dir(phones_tbl.date) if not x.startswith('_')]))

add approx_median approx_nunique arbitrary argmax argmin between bottomk case cases cast coalesce collect compile count cume_dist cummax cummin date day day_of_week day_of_year dense_rank describe epoch_seconds equals execute fillna first get_name greatest group_concat has_name hash hour identical_to isin isnull lag last lead least max millisecond min minute month name notin notnull nth ntile nullif nunique op over parent percent_rank pipe quarter radd rank rsub second strftime sub substitute summary time to_projection topk truncate type typeof value_counts verify visualize week_of_year year


Here are the methods for numeric columns.

In [28]:
print(' '.join([x for x in dir(phones_tbl.overall) if not x.startswith('_')]))

abs acos add approx_median approx_nunique arbitrary argmax argmin asin atan atan2 between bit_and bit_or bit_xor bottomk bucket case cases cast ceil clip coalesce collect compile convert_base corr cos cot count cov cume_dist cummax cummean cummin cumsum deg2rad degrees dense_rank describe div equals execute exp fillna first floor floordiv get_name greatest group_concat has_name hash histogram identical_to isin isnull lag last lead least ln log log10 log2 max mean min mod mul name negate notin notnull nth ntile nullif nullifzero nunique op over parent percent_rank pipe point pow quantile rad2deg radd radians rank rdiv rfloordiv rmod rmul round rpow rsub sign sin sqrt std sub substitute sum summary tan to_interval to_projection to_timestamp topk type typeof value_counts var verify visualize zeroifnull


In [29]:
ibis.options.verbose = False

### Filter reviews containing "iphone 4"

Now that we've seen how to construct expressions that can be used in the database, let's use that expression for use in further queries.
The expression below takes the result of our previous expression, lower-cases it, then checks for the occurrence of the substring
"iphone 4". This will return a column of booleans which can be used to filter our original table, just like with a pandas DataFrame.

In [30]:
has_iphone_4 = texpr.lower().contains('iphone 4')
has_iphone_4

,tmp
0,False
1,False
2,False
3,False
4,False
...,...
3916,False
3917,False
3918,False
3919,False


First, let's use the filter expression to see how many of our generated sentences contain "iphone 4". We do this by using our previous
expression as an indexing expression to our original table object (e.g., `tbl[filter-expr]`). This will return only the rows where
the filter expression was `True`. We can then count the number of rows using the `count` method.

In [31]:
phones_tbl[has_iphone_4].count()

7

Now we can print selected columns from the filtered table. We can also sort them according to the `overall` column as shown below.

In [32]:
phones_tbl[has_iphone_4]['reviewerName', 'helpful', 'overall'].sort_by(ibis.desc('overall'))

,reviewerName,helpful,overall
0,patrice m rogoza,"[2, 3]",5
1,Stephan S.,"[2, 2]",5
2,Monica,"[0, 0]",5
3,Mufasa,"[6, 6]",5
4,Dennis Dennis in NH,"[1, 1]",5
5,Mike,"[0, 0]",4
6,Mark T.,"[1, 1]",4


So far we've uploaded data to the database using `create_table` and a local DataFrame, we've constructed Python expressions that get translated
to SQL and executed on the server, used those expressions in filtering operations, and selected columns from and sorted the results of
the filtering operation. This covers a small number of the abilities of the table and column objects created by Ibis. For additional methods
on the various types, see the [Ibis API reference](https://ibis-project.org/docs/3.2.0/api/expressions/).

## XML book data example

We are going to look at another example utilitizing XML data now. This will demonstrate integration with user-defined functions (UDF) in the server.

### Define book information in XML

As before, we'll use a local pandas DataFrame to construct a table of data and upload it to the server.

In [33]:
xml_df = pd.DataFrame([
    (1, '<book><author>James McGovern</author><title>XQuery Kick Start</title></book>'),
    (2, '<book><author>Aurelien Geron</author><title>Hands-On Machine Learning</title></book>'),
    (3, '<book><author>Luciano Ramalho</author><title>Fluent Python</title></book>'),
    (4, '<book><author>Scott Chacon</author><title>Pro Git</title></book>'),
], columns=['id', 'xml'])

In [34]:
xml_tbl = conn.create_table('books', xml_df, force=True)
xml_tbl.head()

,id,xml
0,1,<book><author>James McGovern</author><title>XQuery Kick Start</title></book>
1,2,<book><author>Aurelien Geron</author><title>Hands-On Machine Learning</title></book>
2,4,<book><author>Scott Chacon</author><title>Pro Git</title></book>
3,3,<book><author>Luciano Ramalho</author><title>Fluent Python</title></book>


### Build and push `eval_xpath` WASM function

We are going to define a UDF using WASM and the [SingleStore WASM toolkit](https://github.com/singlestore-labs/singlestore-wasm-toolkit).
If you want to develop your own UDFs using WASM, you'll want to 
[check out the tutorial](https://singlestore-labs.github.io/singlestore-wasm-toolkit/html/Tutorial-Overview.html). For our purposes here,
we'll use one of the examples from the toolkit.

The `examples/rust/xpath` directory includes a sample UDF that evaluates XPath expressions against strings containing XML documents.
[XPath](https://www.w3.org/TR/xpath/) is a file path-like syntax that can be used to traverse and extract values from XML content.
To build and install the `eval_xpath` function from the examples, you need to run the following commands.

```
> git clone git@github.com:singlestore-labs/singlestore-wasm-toolkit.git
> cd singlestore-wasm-toolkit/examples/rust/xpath
> cargo wasi build --lib --release
> pushwasm --prompt mysql://root@mate:3306/x_db --wit xpath.wit target/wasm32-wasi/release/xpath.wasm eval_xpath
```

Because the function was created before the UDF existed in the server, you'll need to synchronize the functions
to the client manually. However, any UDFs that exist when the connection is created will already be available.

In [35]:
conn.sync_functions()

We can use the `functions` method on the `show` object to display the information for all of the available functions.

In [36]:
conn.show.functions()

We can use the `create_function` method of the `show` object to display the `CREATE FUNCTION` syntax of the function.

In [37]:
conn.show.create_function('eval_xpath')

OperationalError: 1347: 'x_db.eval_xpath' is not a FUNCTION

### Using the UDF

Since the first parameter of the `eval_xpath` is a string, the function can be called on string columns in the table object.
In the code below, we are using the XPath expression `/book/author` to pull the author entry out of each of the XML
documents in the `xml` column. This will return a string expression which we then use to create a filtering expression
by looking for a capital letter 'G' in the author name returned by the XPath expression.

In [38]:
g_authors = xml_tbl.xml.eval_xpath('/book/author').contains('G')
g_authors

AttributeError: 'StringColumn' object has no attribute 'eval_xpath'

As you can see in the result above, two of the author's names contained a capital 'G'. We will now use that filtering expression
to subset the rows in the `xml_tbl` object and return the title of each of the books associated with those authors.

In [40]:
title = xml_tbl[g_authors].xml.eval_xpath('/book/title').name('Title')
title

NameError: name 'g_authors' is not defined